In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('employeeapp').getOrCreate()

In [4]:
df=spark.read.csv("f:\\datasets\\employees.csv",header=True,inferSchema=True)
df.show(5)
print(df.count())
df.printSchema()

+----------+------+----------+---------------+------+------+----------------+---------------+
|First Name|Gender|Start Date|Last Login Time|Salary| Bonus|seniorManagement|           Team|
+----------+------+----------+---------------+------+------+----------------+---------------+
|   Douglas|  Male|      null|           null|  null|  null|            null|      Marketing|
|    Thomas|  Male| 3/31/1996|        6:53 AM| 61933|  4.17|            true|           null|
|     Maria|  null| 4/23/1993|       11:17 AM|  null|11.858|           false|        Finance|
|     Jerry|  Male|03-04-2005|           null|  null|  null|            true|        Finance|
|     Larry|  Male| 1/24/1998|        4:47 PM|  null| 1.389|            true|Client Services|
+----------+------+----------+---------------+------+------+----------------+---------------+
only showing top 5 rows

1000
root
 |-- First Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Start Date: string (nullable = tru

In [5]:
dff=df.na.drop(how='any')
print(dff.count())
dff.show()

756
+----------+------+----------+---------------+------+------+----------------+--------------------+
|First Name|Gender|Start Date|Last Login Time|Salary| Bonus|seniorManagement|                Team|
+----------+------+----------+---------------+------+------+----------------+--------------------+
|     Julie|Female|10/26/1997|        3:19 PM|102508|12.637|            true|               Legal|
|   Brandon|  Male|12-01-1980|        1:08 AM|112807|17.492|            true|     Human Resources|
|      Gary|  Male| 1/27/2008|       11:40 PM|109831| 5.831|           false|               Sales|
|  Kimberly|Female| 1/14/1999|        7:13 AM| 41426|14.543|            true|             Finance|
|   Lillian|Female|06-05-2016|        6:09 AM| 59414| 1.256|           false|             Product|
|    Jeremy|  Male| 9/21/2010|        5:56 AM| 90370| 7.369|           false|     Human Resources|
|     Shawn|  Male|12-07-1986|        7:45 PM|111737| 6.414|           false|             Product|
|     

In [57]:
dff.select('seniorManagement').distinct().show()
dff.printSchema()


+----------------+
|seniorManagement|
+----------------+
|            true|
|           false|
+----------------+

root
 |-- First Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Start Date: string (nullable = true)
 |-- Last Login Time: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Bonus: double (nullable = true)
 |-- seniorManagement: boolean (nullable = true)
 |-- Team: string (nullable = true)



In [58]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
va=VectorAssembler(inputCols=['Salary','Bonus'],outputCol='inputfeat')


In [60]:
df1=va.transform(dff)

In [61]:
df1.show()

+----------+------+----------+---------------+------+------+----------------+--------------------+-----------------+
|First Name|Gender|Start Date|Last Login Time|Salary| Bonus|seniorManagement|                Team|        inputfeat|
+----------+------+----------+---------------+------+------+----------------+--------------------+-----------------+
|     Julie|Female|10/26/1997|        3:19 PM|102508|12.637|            true|               Legal|[102508.0,12.637]|
|   Brandon|  Male|12-01-1980|        1:08 AM|112807|17.492|            true|     Human Resources|[112807.0,17.492]|
|      Gary|  Male| 1/27/2008|       11:40 PM|109831| 5.831|           false|               Sales| [109831.0,5.831]|
|  Kimberly|Female| 1/14/1999|        7:13 AM| 41426|14.543|            true|             Finance| [41426.0,14.543]|
|   Lillian|Female|06-05-2016|        6:09 AM| 59414| 1.256|           false|             Product|  [59414.0,1.256]|
|    Jeremy|  Male| 9/21/2010|        5:56 AM| 90370| 7.369|    

In [62]:


c=df1.withColumn('seniorManagement',df1['seniorManagement'].cast('string'))
c.printSchema()


root
 |-- First Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Start Date: string (nullable = true)
 |-- Last Login Time: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Bonus: double (nullable = true)
 |-- seniorManagement: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- inputfeat: vector (nullable = true)



In [63]:
indexer=StringIndexer(inputCol='seniorManagement',outputCol='senior')
ind=indexer.fit(c).transform(c)
ind.show()

+----------+------+----------+---------------+------+------+----------------+--------------------+-----------------+------+
|First Name|Gender|Start Date|Last Login Time|Salary| Bonus|seniorManagement|                Team|        inputfeat|senior|
+----------+------+----------+---------------+------+------+----------------+--------------------+-----------------+------+
|     Julie|Female|10/26/1997|        3:19 PM|102508|12.637|            true|               Legal|[102508.0,12.637]|   1.0|
|   Brandon|  Male|12-01-1980|        1:08 AM|112807|17.492|            true|     Human Resources|[112807.0,17.492]|   1.0|
|      Gary|  Male| 1/27/2008|       11:40 PM|109831| 5.831|           false|               Sales| [109831.0,5.831]|   0.0|
|  Kimberly|Female| 1/14/1999|        7:13 AM| 41426|14.543|            true|             Finance| [41426.0,14.543]|   1.0|
|   Lillian|Female|06-05-2016|        6:09 AM| 59414| 1.256|           false|             Product|  [59414.0,1.256]|   0.0|
|    Jer

In [64]:
finaldata=ind.select('inputfeat','senior')

In [65]:
finaldata.show()

+-----------------+------+
|        inputfeat|senior|
+-----------------+------+
|[102508.0,12.637]|   1.0|
|[112807.0,17.492]|   1.0|
| [109831.0,5.831]|   0.0|
| [41426.0,14.543]|   1.0|
|  [59414.0,1.256]|   0.0|
|  [90370.0,7.369]|   0.0|
| [111737.0,6.414]|   0.0|
|[132940.0,19.082]|   0.0|
|  [81014.0,1.894]|   0.0|
|[100612.0,13.645]|   0.0|
| [97950.0,13.873]|   0.0|
|  [37598.0,7.757]|   1.0|
|[124008.0,13.464]|   1.0|
|  [79529.0,7.008]|   1.0|
| [118780.0,9.096]|   1.0|
| [119082.0,16.18]|   0.0|
| [95734.0,19.096]|   0.0|
| [85182.0,16.675]|   0.0|
|[142032.0,12.599]|   0.0|
|  [57427.0,9.557]|   1.0|
+-----------------+------+
only showing top 20 rows



In [66]:
train,test=finaldata.randomSplit([.70,.30])

In [67]:
from pyspark.ml.classification  import DecisionTreeClassifier
dtm=DecisionTreeClassifier(labelCol='senior',featuresCol='inputfeat')

In [68]:
model=dtm.fit(train)
print(model)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_16d62c24db6f, depth=5, numNodes=39, numClasses=2, numFeatures=2


In [69]:
pre_res=model.transform(test)

In [70]:
pre_res.select('inputfeat','senior','prediction').show()

+----------------+------+----------+
|       inputfeat|senior|prediction|
+----------------+------+----------+
|[35575.0,14.595]|   0.0|       0.0|
|[35884.0,17.667]|   0.0|       0.0|
|  [36067.0,9.64]|   1.0|       0.0|
|[36711.0,19.028]|   1.0|       0.0|
|[36759.0,17.269]|   1.0|       0.0|
| [36837.0,14.66]|   0.0|       0.0|
| [36946.0,6.652]|   0.0|       0.0|
| [37259.0,1.763]|   0.0|       0.0|
| [37598.0,7.757]|   1.0|       1.0|
| [37812.0,3.178]|   0.0|       0.0|
| [37888.0,13.47]|   0.0|       0.0|
|[37916.0,12.929]|   1.0|       0.0|
| [38092.0,5.668]|   0.0|       0.0|
|[39335.0,10.664]|   0.0|       0.0|
| [39371.0,4.068]|   0.0|       0.0|
| [39833.0,9.631]|   0.0|       0.0|
| [40121.0,6.293]|   0.0|       0.0|
|[41426.0,14.543]|   1.0|       0.0|
|[41808.0,19.239]|   0.0|       0.0|
| [41831.0,4.548]|   0.0|       0.0|
+----------------+------+----------+
only showing top 20 rows



In [42]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [71]:
ev=MulticlassClassificationEvaluator(labelCol='senior',predictionCol='prediction')
acc=ev.evaluate(pre_res)
print('Accuaracy of model:',acc)
print('Test Error of model:',(1-acc))

Accuaracy of model: 0.5135193572899794
Test Error of model: 0.48648064271002056
